In [48]:
#Below are a couple of installment cells that ensure our interactive widgets are correctly imported, these must be ran before using our model code

In [49]:
pip install ipywidgets

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [50]:
from ipywidgets import interact, FloatSlider, IntSlider

In [54]:
import numpy as np
import matplotlib.pyplot as plt

#Below is the function defining the Michaelis-Menten Model's parameters and equation 
def michaelis_menten(S, Vmax, Km):
    """
Equation:
    - v = (Vmax[S])/(Km+[S])
    
Parameters:
    - S: Substrate concentration (This can be set as array or scalar)
    - Vmax: Maximum reaction velocity
    - Km: Michaelis constant, which is the concentration of a given enzyme that catalyzes our reaction
    
Returns:
    - v: Reaction velocity (Or rate of reaction for the Michaelis-Menten Model)
    """
    
    return (Vmax * S) / (Km + S)

#Similarly, the Competitive Inhibition Model's parameters and equation are defined in the funciton below
def competitive_inhibition(S, Vmax, Km, I, Ki):
    """
Equation: 
    - v = (Vmax[S])/(Km(1+([I]/Ki))+[S])
    
Parameters:
    - S: Substrate concentration (This can be set as array or scalar)
    - Vmax: Maximum reaction velocity
    - Km: Michaelis Constant, which is the concentration of a given enzyme that catalyzes our reaction
    - I: Inhibitor Concentration, which represents the total concentration of a present inhibitor 
    - Ki: Inhibition Constant, which measures the concentration of the inhibitor

Returns:
    - v: Reaction velocity (Rate of reaction for the Competitive Inhibition Model)
    """
    
    return (Vmax * S) / (Km * (1 + (I / Ki)) + S)

#Also similarly, the Non-Competitive Inhibition Model's parameters and equation are defined in the funciton below
def non_competitive_inhibition(S, Vmax, Km, I, Ki):
    """
Equation:
    - v = (Vmax[S])/((Km+[S])(1+([I]/Ki)))
    
Parameters (Same as Competitive Inhibition):
    - S: Substrate concentration (This can be set as array or scalar)
    - Vmax: Maximum reaction velocity
    - Km: Michaelis Constant, which is the concentration of a given enzyme that catalyzes our reaction
    - I: Inhibitor Concentration, which represents the total concentration of a present inhibitor 
    - Ki: Inhibition Constant, which measures the concentration of the inhibitor

Returns:
    - v: Reaction velocity (Rate of reaction for the Competitive Inhibition Model)
    """

    return (Vmax * S) / ((Km + S) * (1 + (I / Ki)))

#Below is a description of the Gaussian model of effect, this equation uses pH and temperature to demonstrate an effect on a catalyzed reaction
def adjust_vmax_temp_pH(temp, pH, ideal_temp, ideal_pH, base_Vmax):
    """
    This allows for Vmax to be adjusted based on temperature and pH as inputs.
    We do this using the Gaussian model of effect for tempterature and pH where: effect = e-((x-xideal)^2/(2σ^2)). 
    - x is the current condition 
    - xideal is the ideal value for condition
    - σ is the assumed standard deviation
    """
    temp_effect = np.exp(-((temp - ideal_temp)**2)/(2 * 10**2)) 

    #the assumed Standard Deviation above is 10

    pH_effect = np.exp(-((pH - ideal_pH)**2)/(2 * 1**2))

    #the assumed Standard Deviation above is 1

    return base_Vmax * temp_effect * pH_effect

def adjust_km_temp_pH(temp, pH, ideal_temp, ideal_pH, base_Km):
    """
    This allows for Km to be adjusted based on temperature and pH as inputs.
    This is also done using the Gaussian effect for temperature and pH.
    """
    temp_effect = np.exp(-((temp - ideal_temp)**2)/(2 * 10**2))

    #the assumed Standard Deviation above is 10
    
    pH_effect = np.exp(-((pH - ideal_pH)**2)/(2 * 1**2))
    
    #the assumed Standard Deviation above is 1
    
    return base_Km * (1/(temp_effect * pH_effect))

def plot_models(temp, pH, S_max, I, Ki):
    
#Now we will plot each of our models side by side for easy visualization

    ideal_temp = 37
    #This ^ is the ideal temperature for the reaction in degrees Celcius 
    ideal_pH = 7.4
    #This ^ is the ideal pH for the reaction
    base_Vmax = 100
    #This ^ is the base maximum valocity for the reaction
    base_Km = 50
    #This ^ is the base Michaelis constant for the reaction

    #The below lines adjust Vmax and Km values based on temperature and pH.
    Vmax = adjust_vmax_temp_pH(temp, pH, ideal_temp, ideal_pH, base_Vmax)
    Km = adjust_km_temp_pH(temp, pH, ideal_temp, ideal_pH, base_Km)

    #S is defined as the substrate concetration range 
    S = np.linspace(0, S_max, 500)

    #The below lines calculate the reaction velocity (v) for each different equation/model using our inputs
    v_michaelis = michaelis_menten(S, Vmax, Km)
    v_competitive = competitive_inhibition(S, Vmax, Km, I, Ki)
    v_non_competitive = non_competitive_inhibition(S, Vmax, Km, I, Ki)
    
    #Now we plot all three of our models side by side (using fun squiggly brackets to convert our values to strings)
    fig, axs = plt.subplots(1, 3, figsize=(18, 6))

    #Michaelis-Menten Plot
    axs[0].plot(S, v_michaelis, label= f"Vmax ={Vmax:.2f}, Km ={Km:.2f}")
    axs[0].set_title("Michaelis-Menten Model", fontsize =14)
    axs[0].set_xlabel("Substrate Concentration [S]", fontsize =12)
    axs[0].set_ylabel("Reaction Velocity (v)", fontsize =12)
    axs[0].legend()
    axs[0].grid()

    #Competitive Inhibition Plot
    axs[1].plot(S, v_competitive, label =f"I={I}, Ki ={Ki}, Vmax ={Vmax:.2f}, Km ={Km:.2f}")
    axs[1].set_title("Competitive Inhibition Model", fontsize =14)
    axs[1].set_xlabel("Substrate Concentration [S]", fontsize =12)
    axs[1].set_ylabel("Reaction Velocity (v)", fontsize =12)
    axs[1].legend()
    axs[1].grid()

    #Non-Competitive Inhibition Plot
    axs[2].plot(S, v_non_competitive, label =f"I={I}, Ki ={Ki}, Vmax ={Vmax:.2f}, Km ={Km:.2f}")
    axs[2].set_title("Non-Competitive Inhibition Model", fontsize =14)
    axs[2].set_xlabel("Substrate Concentration [S]", fontsize =12)
    axs[2].set_ylabel("Reaction Velocity (v)", fontsize =12)
    axs[2].legend()
    axs[2].grid()

    plt.tight_layout()
    plt.show()

#lastly, we will create interactive widgets that allow the user to modify values of temperature and pH to observe changes
interact(
    plot_models,
    temp = FloatSlider(value =25, min =0, max =100, step =1, description ='Temperature (Celcius)'),
    #The above line creates a slider ranging from 0 to 100 that allows the user to set the temperature for each model
    
    pH = FloatSlider(value =7, min =0, max =14, step =0.1, description ='pH'),
    #The above line similarly creates a slider ranging from 0 to 14 for the user to modify pH with

    S_max = IntSlider(value =200, min =50, max =500, step =10, description ='S Max'),
    #The above line creates a slider ranging from 50 to 500 that allows the user to set the S Max value for each model

    I = FloatSlider(value =1, min =0, max =10, step =0.1, description ='Inhibitor [I]'),
    #The above line creates a slider ranging from 0 to 10, allowing the user to modify the Inhibitor value for the Inhibition Models

    Ki = FloatSlider(value =5, min =1, max =20, step =0.1, description ='Inhibition Constant (Ki)')
    #The above line creates a slider ranging from 1 to 20 that similarly allows the user to modify the Ki value for the Inhibition Models
)

interactive(children=(FloatSlider(value=25.0, description='Temperature (Celcius)', step=1.0), FloatSlider(valu…

<function __main__.plot_models(temp, pH, S_max, I, Ki)>